In [30]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import LlamafileEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

raw_documents = TextLoader('./apple.txt', encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, LlamafileEmbeddings(), persist_directory="./chroma_db")

query = "What have they done in the garage?"
docs = db.similarity_search(query)
print(docs[0].page_content)


Timmy's eyes sparkled. "That's an excellent idea! But we need a name... something simple, yet powerful." They looked up at the apple tree above them. "How about 'Apple'?" Martina suggested. "An apple stands for curiosity and knowledge. Exactly what we want our computers to represent." Timmy nodded enthusiastically.

They started working in their garage, with old parts and plenty of creativity. Timmy took care of the computer's insides – ensuring it was easy to use yet powerful. Martina, with her sense of aesthetics, designed the computer's exterior to make it friendly and inviting.

Day and night, they worked, supported by their friends and family who believed in their vision. Finally, after many months of hard work, the first "Apple" computer was ready. It was small, affordable, and so simple that truly anyone could use it.
